In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("Ken_French_Data.csv", header=0, index_col=0, na_values=-99.99)

rets = data/100
rets.index = pd.to_datetime(rets.index, format="%Y%m").to_period('M')

#Paper uses excess return, therefore need to take risk-free rate off industry returns
d1 = rets
d1 = d1.drop(['Mkt-RF','RF'], 1)
d1 = d1.sub(rets['RF'], axis =0)
d1["Mkt-RF"] = rets["Mkt-RF"]

rets = d1

# Function Preparation


In [3]:
def monthly_rets(r):
    return r.mean()

def monthly_vol(r):
    return r.std()

def portfolio_return(weights, returns):
    return weights.T @ returns

def portfolio_vol (weights, cov):
    return (weights.T @ cov @ weights)**0.5

#Risk free rate set to 0 as the data is already excess returns
def sharpe_ratio(r):
    sharpe = r.mean()/r.std()
    return sharpe

def weight_ew(r):
    n = len(r.columns)
    ew = pd.Series(1/n, index=r.columns)
    return ew

def mv_func(er, cov):
    num_assets = cov.shape[0]
    inv_cov = np.linalg.pinv(cov)
    weights = (inv_cov@er)/(np.ones((1,num_assets))@inv_cov@er)
    return weights

def min_v_func(cov):
    num_assets = cov.shape[0]
    er = [1] * num_assets
    inv_cov = np.linalg.pinv(cov)
    weights = (inv_cov@er)/(np.ones((1,num_assets))@inv_cov@er)
    return weights


# Equal-weighted Portfolio 

In [4]:
def weight_ew(r):
    n = len(r.columns)
    ew = pd.Series(1/n, index=r.columns)
    return ew

RW = 120

PortRet= []  
for Date,ew in rets[RW:].iterrows():
    ew = weight_ew(rets).values    
    PortRet.append(ew@rets.loc[Date].values)

EW_Result = pd.DataFrame(PortRet, index = rets[RW:].index, columns =['ew'])


# In-sample mv portfolio

In [5]:
#Full sample mv portfolio
f_ret = monthly_rets(rets)
f_cov = rets.cov()

#Generating weights
mv_weights_IS = mv_func(f_ret, f_cov)

#Multiplying weights with industries return
f_Port = rets.mul(mv_weights_IS, axis=1).sum(axis=1)
f_Port =pd.DataFrame(f_Port, index = rets.index, columns =['Full sample mean-variance in-sample'])

In [6]:
#"In-sample" testing with first 50 months of data 
RW = 50
is_ret = rets[:RW]
is_er = monthly_rets(is_ret)
is_cov = rets[:RW].cov()

#Generating weights
c = mv_func(is_er, is_cov)

#Mulipl weights with return
MV_Port_0 = rets[:RW].mul(c, axis=1).sum(axis=1)
MV_Port_0 =pd.DataFrame(MV_Port_0, index = rets[:RW].index, 
                       columns =['Mean-variance in-sample 50-Months'])

In [7]:
#"In-sample" testing with first 120 months of data 
RW = 120
is_ret = rets[:RW]
is_er = monthly_rets(is_ret)
is_cov = rets[:RW].cov()

#Generating weights
c = mv_func(is_er, is_cov)

#Mulipl weights with return
MV_Port_1 = rets[:RW].mul(c, axis=1).sum(axis=1)
MV_Port_1 =pd.DataFrame(MV_Port_1, index = rets[:RW].index, 
                       columns =['Mean-variance in-sample 120-Months'])

# Out-of-sample mean variance portfolio - 120 Rolling Window

In [8]:
#Rolling window for mean and variance
RW_mean = rets.rolling(RW, min_periods= RW).mean().dropna()
RW_cov = rets.rolling(RW, min_periods= RW).cov().dropna()

#Shifting date to match analysis dates as it is t-1
RW_mean = RW_mean.loc["1973-07":"2004-11"]
RW_cov = RW_cov.loc["1973-07": "2004-11"]

In [9]:
#Developing backtest system in a rolling window basis
OoS_MV = []
for Dates, w in RW_cov.groupby(level = "Date"):
    er = RW_mean.loc[Dates]
    cov = RW_cov.loc[Dates,:].values
    w = mv_func(er, cov)

    OoS_MV.append(w)
    
OoS_MV = pd.DataFrame(OoS_MV, index= rets[RW:].index, columns = list(RW_mean))
    
OoS_MV_rets = []
for Date, w in rets[RW:].iterrows():
    w = OoS_MV.loc[Date].values  
    OoS_MV_rets.append(w@rets.loc[Date].values)
    
OoS_MV_rets =pd.DataFrame(OoS_MV_rets, index = rets[RW:].index, columns =['Mean-variance out-of-sample 120-Months'])

# Oos Min-Variance Portfolio

In [10]:
#Developing backtest system in a rolling window basis
OoS_MinV = []
for Dates, w in RW_cov.groupby(level = "Date"):
    cov = RW_cov.loc[Dates,:].values
    w = min_v_func(cov)
    OoS_MinV.append(w)
    
OoS_MinV = pd.DataFrame(OoS_MinV, index= rets[RW:].index, columns = list(RW_mean))
    
OoS_MinV_rets = []
for Date, w in rets[RW:].iterrows():
    w = OoS_MinV.loc[Date].values  
    OoS_MinV_rets.append(w@rets.loc[Date].values)
    
OoS_MinV_rets =pd.DataFrame(OoS_MinV_rets, index = rets[RW:].index, columns =['Min_Var out-of-sample 120-Months'])

# Result of portfolios

In [11]:
btr = pd.DataFrame({"1/N ": EW_Result["ew"],
                    "MV (OoS)": OoS_MV_rets["Mean-variance out-of-sample 120-Months"],
                    "MV (IS)": f_Port['Full sample mean-variance in-sample'],
                    "Min (OoS)": OoS_MinV_rets['Min_Var out-of-sample 120-Months']
                   })

btr.apply(sharpe_ratio, axis = 0)

1/N          0.136496
MV (OoS)     0.076015
MV (IS)      0.210754
Min (OoS)    0.169816
dtype: float64